In [1]:
from data_loader import *
import csv
from pycocotools.coco import COCO
from tqdm import tqdm
from vocab import *
import nltk
import os
import torchvision.transforms as transforms
import torch.nn.functional as TF
import numpy as np
from LSTM import *
import torch.nn as nn
import math
import torch.optim as optim

In [2]:
batch_size = 2
vocab = Vocab()
nltk.download('punkt')

#print(vocab.wordDict[","])
print(vocab.vocabSize)
print(vocab.maxLength)

Successfully loaded vocabulary from wordDict.pkl and wordStat.pkl
29330
50


[nltk_data] Downloading package punkt to /home/dshvarts/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Hyper parameters for LSTM and word embedding.
# Refer to https://pytorch.org/docs/stable/nn.html#lstm for their meaning.
cfg = {
    'input_size': 50, # In our case, it should be the size of the embedded word
    'output_size': 100, # In our case, it should be the size of the one hot encoded word
    'num_layers': 1,
    'dropout': 0,
    'bidirectional': False,
    'embedding_size': 200,
    'vocab_size': vocab.vocabSize,
    'hidden_size': 300,
    'learning_rate': 1e-5
}

In [15]:
annos = './data/annotations/captions_train2014.json'
images = '/datasets/COCO-2015/train2014/'
coco = COCO(annos)
trainIdsNew = []
valIds = []

# get ids of training images
with open('TrainImageIds.csv', 'r') as f:
    reader = csv.reader(f)
    trainIds = list(reader)
trainImgIds = [int(i) for i in trainIds[0]]

trainSet = trainImgIds[:math.floor(len(trainImgIds) * 0.8)]
valSet = trainImgIds[math.floor(len(trainImgIds) * 0.8)+1: ]

print(len(trainSet))
print(len(valSet))

for imgId in trainSet:
    for caption in coco.imgToAnns[imgId]:
        trainIdsNew.append(caption['id'])

        
for imgId in valSet:
    for caption in coco.imgToAnns[imgId]:
        valIds.append(caption['id'])
        
        
train_loader = get_loader(root=images, json=annos, ids=trainIdsNew, vocab=vocab,
                         transform=transforms.Resize((256,256)), batch_size=batch_size, shuffle=True, num_workers=4)

val_loader = get_loader(root=images, json=annos, ids=valIds, vocab=vocab,
                         transform=transforms.Resize((256,256)), batch_size=batch_size, shuffle=True, num_workers=4)



loading annotations into memory...
Done (t=0.47s)
creating index...
index created!
13244
3311
509365 661353
509365 663117
509365 667275
509365 668043
509365 668337
87007 657600
87007 659169
87007 662961
87007 669198
87007 669474
79642 421757
79642 423638
79642 425291
79642 427031
79642 429074
475957 577904
475957 581570
475957 581831
475957 582380
475957 583136
363624 563499
363624 563568
363624 565056
363624 565977
363624 566982
293453 607116
293453 613440
293453 613542
293453 616158
293453 618963
428787 456329
428787 456617
428787 461300
428787 464087
428787 465047
367171 697929
367171 698574
367171 700791
367171 702714
367171 704220
376029 124428
376029 145005
376029 153966
376029 157008
376029 164649
286101 463164
286101 465891
286101 466659
286101 467088
286101 468663
54020 451768
54020 457147
54020 457879
54020 459553
54020 825263
401617 704478
401617 711174
401617 714120
401617 715278
401617 715767
526754 606487
526754 613267
526754 615115
526754 617812
526754 618268
284670 6368

135467 134971
135467 141109
135467 144907
368347 596980
368347 598057
368347 599413
368347 600049
368347 600298
490016 659550
490016 660405
490016 661044
490016 662244
490016 664356
91666 636051
91666 636438
91666 646653
91666 646917
91666 647307
520036 419860
520036 423232
520036 423625
520036 430000
520036 430582
536041 522811
536041 525127
536041 525763
536041 526642
536041 526807
360246 569986
360246 570892
360246 570967
360246 574099
360246 574408
17578 694556
17578 695135
17578 696206
17578 697979
17578 699434
411815 436036
411815 436534
411815 437638
411815 438637
411815 441949
253577 83606
253577 85412
253577 92372
253577 95807
253577 112979
506231 521957
506231 522683
506231 523586
506231 525557
506231 526085
186148 589214
186148 596570
186148 596669
186148 745742
186148 748753
574194 513192
574194 513312
574194 514830
574194 516126
574194 527994
96414 753076
96414 753546
96414 756492
96414 760090
96414 763447
169810 476516
169810 477257
169810 478109
169810 478205
169810 4805

436362 316870
436362 318853
436362 320662
62324 76538
62324 81944
62324 84353
62324 86123
62324 87290
95201 271035
95201 277161
95201 277320
95201 280272
95201 282684
233925 147892
233925 150736
233925 151108
233925 154981
233925 829233
45840 653452
45840 656989
45840 657958
45840 658843
45840 659071
209449 225699
209449 231774
209449 244539
209449 247515
209449 251049
469260 514600
469260 515170
469260 515224
469260 517741
469260 518011
469260 518050
431554 733127
431554 733451
431554 733898
431554 734552
431554 736205
70459 682466
70459 682478
70459 683684
70459 687872
70459 693107
376523 772632
376523 774384
376523 774786
376523 778539
376523 783087
135733 139923
135733 146877
135733 147819
135733 154329
135733 158043
125298 70953
125298 75897
125298 76782
125298 80607
125298 81717
487466 117230
487466 128759
487466 133376
487466 139868
487466 142304
253467 518776
253467 524176
253467 525364
253467 526279
253467 526453
103234 514513
103234 514960
103234 515167
103234 517219
103234 5

562712 180126
578171 691281
578171 692097
578171 692802
578171 695859
578171 699570
360185 272703
360185 273006
360185 273789
360185 276450
360185 276825
157965 712465
157965 712618
157965 712690
157965 713350
157965 715450
224878 384339
224878 385008
224878 389853
224878 391812
224878 392649
532310 142131
532310 145020
532310 146163
532310 149718
532310 150423
246285 320100
246285 321075
246285 321240
246285 325425
246285 326106
486250 535474
486250 536059
486250 538123
486250 540640
486250 545536
262407 734159
262407 734219
262407 736376
262407 736610
262407 738788
483451 293685
483451 294561
483451 296286
483451 297564
483451 298749
186777 41981
186777 42515
186777 48227
186777 48338
186777 48422
11544 339765
11544 340749
11544 344646
11544 344655
11544 344940
209844 767406
209844 768939
209844 772371
209844 772914
209844 772956
25374 788580
25374 792708
25374 793542
25374 797181
25374 798303
107001 260495
107001 261095
107001 261215
107001 264545
107001 265847
466822 498349
466822 

81363 371561
465603 341261
465603 343997
465603 344345
465603 344384
465603 344720
466580 288616
466580 292681
466580 293110
466580 296485
466580 297850
364806 187210
364806 198229
364806 199435
364806 204244
364806 206266
11682 1944
11682 2142
11682 2697
11682 3018
11682 5817
468382 146150
468382 180998
468382 193496
468382 194708
468382 220922
562037 24092
562037 24140
562037 31988
562037 34889
562037 37724
486004 76484
486004 77162
486004 78941
486004 79901
486004 81404
251671 623424
251671 629148
251671 633762
251671 635091
251671 636540
9460 40665
9460 43398
9460 47199
9460 50679
9460 59604
285512 749899
285512 750493
285512 752220
285512 753027
285512 754592
66858 379326
66858 382038
66858 382419
66858 383424
66858 385374
81602 237221
81602 248183
81602 258026
81602 258206
81602 260150
56652 670557
56652 672561
56652 675657
56652 678963
56652 679518
265279 471558
265279 475938
265279 478350
265279 480258
265279 821821
238772 573462
238772 576612
238772 577458
238772 578073
238772

...

289569 197523
289569 200865
289569 220350
456648 491411
456648 493355
456648 493901
456648 494999
456648 498503
313768 42570
313768 46404
313768 47232
313768 48969
313768 51000
98233 432771
98233 433020
98233 433254
98233 433761
98233 435306
57515 394863
57515 396726
57515 396801
57515 398964
57515 401007
256075 485383
256075 486946
256075 488134
256075 490963
256075 827864
272463 457168
272463 463030
272463 463768
272463 469813
272463 820688
565518 369240
565518 369984
565518 371076
565518 371931
565518 380796
238881 475018
238881 477169
238881 481162
238881 483229
238881 484891
99247 26176
99247 26320
99247 26638
99247 26782
99247 34744
238960 169580
238960 174317
238960 175313
238960 177221
238960 180119
333181 227768
333181 264281
333181 265439
333181 265511
333181 276785
356402 756844
356402 761963
356402 763685
356402 765494
356402 765983
127028 1330
127028 1507
127028 6004
127028 8929
127028 12274
86213 730124
86213 731033
86213 732518
86213 734606
86213 734873
181064 598230
181

463738 539001
463738 541188
463738 542688
463738 549690
463738 551337
466623 599516
466623 601592
466623 601889
466623 607400
466623 613385
535552 469206
535552 476271
535552 482235
535552 483216
535552 483909
185368 1847
185368 4196
185368 5168
185368 8588
185368 14483
156701 215681
156701 219233
156701 224825
156701 228929
156701 229619
406920 450782
406920 458381
406920 461099
406920 465623
406920 822607
229236 224486
229236 248045
229236 248927
229236 257672
229236 262277
4888 59600
4888 60380
4888 65087
4888 65660
4888 67865
475888 625421
475888 625427
475888 626318
475888 632210
475888 634109
262704 429280
262704 430165
262704 430717
262704 431089
262704 432331
80386 298884
80386 298896
80386 299214
80386 300567
80386 301590
179940 348340
179940 349741
179940 362557
179940 362743
179940 367096
430790 76563
430790 82794
430790 83292
430790 83646
430790 83949
323661 448683
323661 449247
323661 451896
323661 457068
323661 820005
484992 431173
484992 432982
484992 437338
484992 44047

219141 264655
333831 508310
333831 510830
333831 511277
333831 512408
333831 549794
409652 268847
409652 273455
409652 273680
409652 276518
409652 276614
275708 782184
275708 783030
275708 784056
275708 784941
275708 787398
426031 738461
426031 738935
426031 739205
426031 740753
426031 741467
16697 233173
16697 242236
16697 247957
16697 256876
16697 258160
39017 141888
39017 143013
39017 146955
39017 149028
39017 149466
576931 181527
576931 190446
576931 192768
576931 193617
576931 201540
122874 312122
122874 312158
122874 312209
122874 312647
122874 313703
453558 695492
453558 695906
453558 697049
453558 698084
453558 699782
322349 20631
322349 21384
322349 23835
322349 26181
322349 26541
319339 162076
319339 165604
319339 166837
319339 168295
319339 168754
252272 394944
252272 395469
252272 395961
252272 399315
252272 519900
147371 111062
147371 119717
147371 121487
147371 121736
147371 139310
474453 734433
474453 735966
474453 736167
474453 738489
474453 821156
130314 799407
130314 

504348 275453
32937 755930
32937 757661
32937 758497
32937 760602
32937 761316
526523 307147
526523 307534
526523 311110
526523 312205
526523 312298
257858 518927
257858 520124
257858 521903
257858 528440
257858 528608
141852 705972
141852 706425
141852 709251
141852 711201
141852 711693
492550 302990
492550 304439
492550 307637
492550 307898
492550 308669
206685 148736
206685 195341
206685 198296
206685 199676
206685 217121
188405 501468
188405 501534
188405 503844
188405 504423
188405 505788
197690 124165
197690 173824
197690 182302
197690 184702
197690 185719
245853 333836
245853 338138
245853 341615
245853 344282
245853 345038
415909 780748
415909 781216
415909 781387
415909 785023
415909 787213
275932 67310
275932 71273
275932 71663
275932 72506
275932 72920
254629 800624
254629 800654
254629 802793
254629 806801
254629 807494
378055 517880
378055 519440
378055 522791
378055 526793
378055 528395
305557 88332
305557 89151
305557 92655
305557 95166
305557 95253
351359 113063
351359 

Done (t=0.47s)
creating index...
index created!
loading annotations into memory...
Done (t=0.46s)
creating index...
index created!


In [5]:
annos = './data/annotations/captions_val2014.json'
images = '/datasets/COCO-2015/val2014/'
coco = COCO(annos)
testIds = []

# get ids of test
with open('TestImageIds.csv', 'r') as f:
    reader = csv.reader(f)
    testIds = list(reader)
testImgIds = [int(i) for i in testIds[0]]

for imgId in testImgIds:
    for caption in coco.imgToAnns[imgId]:
        testIds.append(caption['id'])

test_loader = get_loader(root=images, json=annos, ids=testIds, vocab=vocab,
                         transform=transforms.Resize((256,256)), batch_size=batch_size, shuffle=True, num_workers=4)

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!
loading annotations into memory...
Done (t=0.23s)
creating index...
index created!


In [6]:
model = LSTM(cfg).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=cfg["learning_rate"])

In [7]:
for iter, (images, targets, lengths) in enumerate(train_loader):
    images = images.cuda()
    
    '''one_hot_target = TF.one_hot(targets, num_classes=vocab.vocabSize)
    one_hot_target = one_hot_target.float()'''
    targets = targets.cuda()
    #print(one_hot_target.shape)

    Y, (_, _) = model(X=targets, imgs=images)

    

    
    Y = Y[:,:-1,:]
#     print("input")
#     print(Y.shape)
#     print(Y)
#     print("targets")
#     print(targets.shape)
    
    #print(targets)
    
    Y = Y.reshape((Y.shape[0] * Y.shape[1], Y.shape[2]))
    targets = targets.reshape(targets.shape[0] * targets.shape[1])

    
    loss = criterion(Y.float(), targets.long())
    loss.backward()
    optimizer.step()
    print(loss)


tensor(10.2930, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2381, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2668, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2841, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2681, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2561, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2628, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2747, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2732, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2947, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2762, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2904, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2799, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2758, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2600, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2309, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2486, device='cuda:0', grad_fn

tensor(10.1339, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.1394, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.0844, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.0952, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.0493, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.0041, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.1218, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.1826, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.1857, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.8972, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.8612, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.1211, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.9577, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.1197, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.1025, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.1037, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.0796, device='cuda:0', grad_fn=<N

tensor(9.8988, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.1221, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.2811, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.2127, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.9361, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.9252, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.6071, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.3596, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.7948, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.8675, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.8282, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.4755, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.8125, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.7429, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.4001, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.9296, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.9124, device='cuda:0', grad_fn=<NllLossBackwar

tensor(8.5594, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.9032, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.1626, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.8966, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.6774, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.4201, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.2869, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.4274, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.6578, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.2267, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.8818, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.2979, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.0766, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.5293, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.0768, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.4919, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.5863, device='cuda:0', grad_fn=<NllLossBackward

tensor(6.7173, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.1236, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.7911, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.9319, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.4754, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.2677, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.6892, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.4784, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.5658, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.2321, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.9161, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.0571, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.5303, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.7111, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.3154, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.9187, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.6209, device='cuda:0', grad_fn=<NllLossBackward

...

tensor(3.4748, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.6410, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.3539, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.6831, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.7980, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.4477, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.7455, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.2426, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.1574, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.4706, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.7740, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.5436, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.9658, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.8538, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.2506, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.4479, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.9036, device='cuda:0', grad_fn=<NllLossBackward

tensor(4.0981, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.0538, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.3774, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.5113, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.7218, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.9813, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.5034, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.8858, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.2550, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1622, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.8693, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.4080, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.7347, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.5835, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.4004, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.6093, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.8480, device='cuda:0', grad_fn=<NllLossBackward

tensor(3.9668, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.7624, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.6660, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.9845, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.0738, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.6638, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.1457, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.2011, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.3627, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.6329, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.7216, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.9692, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.7535, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.4380, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.9070, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.4952, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.9760, device='cuda:0', grad_fn=<NllLossBackward

tensor(5.4571, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.2509, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.3807, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.8838, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.7012, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.2285, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.1585, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.5097, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.6763, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.6486, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.5669, device='cuda:0', grad_fn=<NllLossBackward>)


TypeError: Caught TypeError in DataLoader worker process 3.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/dshvarts/cse253_pa4/data_loader.py", line 38, in __getitem__
    img_id = coco.anns[ann_id]['image_id']
TypeError: unhashable type: 'list'


In [ ]:
'''# find the min width and height to crop
annos = './data/annotations/captions_train2014.json'
root = '/datasets/COCO-2015/train2014/'
coco = COCO(annos)
minWidth = 1000000
minHeight = 1000000
for imgId in trainImgIds:
    path = coco.loadImgs(imgId)[0]['file_name']
    image = Image.open(os.path.join(root, path)).convert('RGB')
    width, height = image.size
    minWidth = min(minWidth, width)
    minHeight = min(minHeight, height)'''

In [ ]:
'''print(minWidth, minHeight)'''

In [ ]:
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.tensor([1, 0, 4])
output = loss(m(input), target)

print(input)
print(target)

print(target.shape)
print(input.shape)

['509365', '87007', '79642', '475957', '363624', '293453', '428787', '367171', '376029', '286101', '54020', '401617', '526754', '284670', '465066', '35045', '319766', '497621', '502805', '289684', '570358', '128022', '402563', '21329', '79085', '123568', '479462', '129071', '408978', '114068', '462110', '391401', '79080', '491272', '291004', '149849', '573854', '445143', '257847', '282097', '227734', '333910', '492069', '461521', '339918', '524029', '71502', '134128', '12416', '345668', '414888', '260603', '335683', '486533', '357169', '576653', '571690', '110567', '153105', '543817', '185151', '295265', '235466', '176521', '381788', '39779', '378768', '30355', '563798', '289031', '407582', '474155', '483833', '424105', '229525', '159331', '290899', '91935', '223932', '151054', '401603', '151277', '496678', '176179', '33732', '577246', '110187', '191706', '28547', '510006', '194694', '344996', '436214', '87812', '188148', '226974', '256057', '336107', '562911', '73554', '167431', '1530